[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1UvR9m2KTinvlDXeOWfS2HBU4X72LAjTz?usp=sharing)

# Llama Stack Benchmark Evals

This notebook will walk you through the main sets of APIs we offer with Llama Stack for supporting running benchmark evaluations of your with working examples to explore the possibilities that Llama Stack opens up for you.

Read more about Llama Stack: https://llama-stack.readthedocs.io/en/latest/index.html

## 0. Bootstrapping Llama Stack Library

##### 0.1. Prerequisite: Create TogetherAI account

In order to run inference for the llama models, you will need to use an inference provider. Llama stack supports a number of inference [providers](https://github.com/meta-llama/llama-stack/tree/main/llama_stack/providers/remote/inference).

In this showcase, we will use [together.ai](https://www.together.ai/) as the inference provider. So, you would first get an API key from Together if you dont have one already.
You can also use Fireworks.ai or even Ollama if you would like to.


> **Note:**  Set the API Key in the Secrets of this notebook as `TOGETHER_API_KEY`

In [ ]:
!pip install -U llama-stack

In [ ]:
!llama stack build --template together --image-type venv

Installing pip dependencies
sentence-transformers --no-deps
torch --index-url https://download.pytorch.org/whl/cpu
Looking in indexes: https://download.pytorch.org/whl/cpu
Build Successful!


In [ ]:
import os

from google.colab import userdata

os.environ["TOGETHER_API_KEY"] = userdata.get("TOGETHER_API_KEY")

from llama_stack.distribution.library_client import LlamaStackAsLibraryClient

client = LlamaStackAsLibraryClient("together")
_ = client.initialize()

# register 405B as LLM Judge model
client.models.register(
    model_id="meta-llama/Llama-3.1-405B-Instruct",
    provider_model_id="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo",
    provider_id="together",
)


Using config together:

apis:
- agents
- datasetio
- eval
- inference
- memory
- safety
- scoring
- telemetry
conda_env: together
datasets: []
docker_image: null
eval_tasks: []
image_name: together
memory_banks: []
metadata_store:
  db_path: /root/.llama/distributions/together/registry.db
  namespace: null
  type: sqlite
models:
- metadata: {}
  model_id: meta-llama/Llama-3.1-8B-Instruct
  model_type: &id001 !!python/object/apply:llama_stack.apis.models.models.ModelType
  - llm
  provider_id: null
  provider_model_id: meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.1-70B-Instruct
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.1-405B-Instruct-FP8
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-3B-Instruct
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Llama-3.2-3B-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-11B-Vision-Instruct
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Llama-3.2-11B-Vision-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-3.2-90B-Vision-Instruct
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo
- metadata: {}
  model_id: meta-llama/Llama-Guard-3-8B
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Meta-Llama-Guard-3-8B
- metadata: {}
  model_id: meta-llama/Llama-Guard-3-11B-Vision
  model_type: *id001
  provider_id: null
  provider_model_id: meta-llama/Llama-Guard-3-11B-Vision-Turbo
providers:
  agents:
  - config:
      persistence_store:
        db_path: /root/.llama/distributions/together/agents_store.db
        namespace: null
        type: sqlite
    provider_id: meta-reference
    provider_type: inline::meta-reference
  datasetio:
  - config: {}
    provider_id: huggingface
    provider_type: remote::huggingface
  - config: {}
    provider_id: localfs
    provider_type: inline::localfs
  eval:
  - config: {}
    provider_id: meta-reference
    provider_type: inline::meta-reference
  inference:
  - config:
      api_key: 4985b03e627419b2964d34b8519ac6c4319f094d1ffb4f45514b4eb87e5427a2
      url: https://api.together.xyz/v1
    provider_id: together
    provider_type: remote::together
  memory:
  - config:
      kvstore:
        db_path: /root/.llama/distributions/together/faiss_store.db
        namespace: null
        type: sqlite
    provider_id: faiss
    provider_type: inline::faiss
  safety:
  - config: {}
    provider_id: llama-guard
    provider_type: inline::llama-guard
  scoring:
  - config: {}
    provider_id: basic
    provider_type: inline::basic
  - config: {}
    provider_id: llm-as-judge
    provider_type: inline::llm-as-judge
  - config:
      openai_api_key: ''
    provider_id: braintrust
    provider_type: inline::braintrust
  telemetry:
  - config:
      service_name: llama-stack
      sinks: sqlite
      sqlite_db_path: /root/.llama/distributions/together/trace_store.db
    provider_id: meta-reference
    provider_type: inline::meta-reference
scoring_fns: []
shields:
- params: null
  provider_id: null
  provider_shield_id: null
  shield_id: meta-llama/Llama-Guard-3-8B
version: '2'

Model(identifier='meta-llama/Llama-3.1-405B-Instruct', metadata={}, provider_id='together', provider_resource_id='meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo', type='model', model_type='llm')

## 1. Open Benchmark Model Evaluation

The first example walks you through how to evaluate a model candidate served by Llama Stack on open benchmarks. We will use the following benchmark:

- [MMMU](https://arxiv.org/abs/2311.16502) (A Massive Multi-discipline Multimodal Understanding and Reasoning Benchmark for Expert AGI)]: Benchmark designed to evaluate multimodal models.
- [SimpleQA](https://openai.com/index/introducing-simpleqa/): Benchmark designed to access models to answer short, fact-seeking questions.

#### 1.1 Running MMMU
- We will use a pre-processed MMMU dataset from [llamastack/mmmu](https://huggingface.co/datasets/llamastack/mmmu). The preprocessing code is shown in in this [Github Gist](https://gist.github.com/yanxi0830/118e9c560227d27132a7fd10e2c92840). The dataset is obtained by transforming the original [MMMU/MMMU](https://huggingface.co/datasets/MMMU/MMMU) dataset into correct format by `inference/chat-completion` API.

In [ ]:
name = "llamastack/mmmu"
subset = "Agriculture"
split = "dev"


In [ ]:
import datasets

ds = datasets.load_dataset(path=name, name=subset, split=split)
ds = ds.select_columns(["chat_completion_input", "input_query", "expected_answer"])
eval_rows = ds.to_pandas().to_dict(orient="records")


README.md:   0%|          | 0.00/36.0k [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/29.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/165M [00:00<?, ?B/s]

test-00000-of-00003.parquet:   0%|          | 0.00/461M [00:00<?, ?B/s]

test-00001-of-00003.parquet:   0%|          | 0.00/454M [00:00<?, ?B/s]

test-00002-of-00003.parquet:   0%|          | 0.00/471M [00:00<?, ?B/s]

Generating dev split:   0%|          | 0/5 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/30 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/287 [00:00<?, ? examples/s]

- **Run Evaluation on Model Candidate**
  - Define a System Prompt
  - Define an EvalCandidate
  - Run evaluate on datasets

In [ ]:
from rich.pretty import pprint
from tqdm import tqdm

SYSTEM_PROMPT_TEMPLATE = """
You are an expert in {subject} whose job is to answer questions from the user using images.

First, reason about the correct answer.

Then write the answer in the following format where X is exactly one of A,B,C,D:

Answer: X

Make sure X is one of A,B,C,D.

If you are uncertain of the correct answer, guess the most likely one.
"""

system_message = {
    "role": "system",
    "content": SYSTEM_PROMPT_TEMPLATE.format(subject=subset),
}

client.eval_tasks.register(
    eval_task_id="meta-reference::mmmu",
    dataset_id=f"mmmu-{subset}-{split}",
    scoring_functions=["basic::regex_parser_multiple_choice_answer"],
)

response = client.eval.evaluate_rows(
    task_id="meta-reference::mmmu",
    input_rows=eval_rows,
    scoring_functions=["basic::regex_parser_multiple_choice_answer"],
    task_config={
        "type": "benchmark",
        "eval_candidate": {
            "type": "model",
            "model": "meta-llama/Llama-3.2-90B-Vision-Instruct",
            "sampling_params": {
                "strategy": {
                    "type": "greedy",
                },
                "max_tokens": 4096,
                "repeat_penalty": 1.0,
            },
            "system_message": system_message,
        },
    },
)
pprint(response)


100%|██████████| 5/5 [00:51<00:00, 10.28s/it]


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'generated_answer': 'The Colorado potato beetle (Leptinotarsa decemlineata) is a significant pest of potatoes, causing damage to the leaves and stems of potato plants. The insect with black-colored antennae in the image is a Colorado potato beetle, which is known for its distinctive black and yellow stripes. On the other hand, the insect with tan-colored antennae is not a Colorado potato beetle and does not appear to be a pest of potatoes.\n\n*Answer*: B) The one with black coloured antennae'
│   │   },
│   │   {
│   │   │   'generated_answer': 'To determine the count of pathogens infecting this sunflower leaf, we need to analyze the image carefully. The image shows a sunflower leaf with several brown spots and patches on its surface. These brown spots and patches are indicative of fungal infections, which are common pathogens that affect sunflowers.\n\nUpon closer inspection, we can see that there are two distinct types of brown spots and patches on the leaf. One type is smaller and more circular in shape, while the other type is larger and more irregular in shape. This suggests that there may be two different pathogens infecting the leaf.\n\nHowever, without further information or testing, it is difficult to say for certain whether these two types of brown spots and patches are caused by different pathogens or if they are just different stages of the same infection. Therefore, based on the available information, the most likely answer is:\n\nAnswer: B) Two pathogens'
│   │   },
│   │   {
│   │   │   'generated_answer': 'Based on the image, the most likely reason for the massive gum production on the trunks of these grapefruit trees in Cyprus is a fungal infection. The gummosis, or the production of gum, is a common symptom of fungal diseases in citrus trees, and it can be caused by various factors such as root damage, water stress, or nutrient deficiencies. However, in this case, the presence of the gum on the trunks of the trees suggests that the cause is more likely related to a fungal infection.\n\nAnswer: E) Fungal gummosis'
│   │   },
│   │   {
│   │   │   'generated_answer': 'The correct answer is D) Most viruses have a specific relationship with their vectors.\n\nExplanation:\n\n*   Laboratory work with micro manipulators can mimic the transmission of viruses, but this is not the primary method of virus transmission in nature.\n*   Not all plant-feeding insects can transmit viruses; only specific species that have evolved to transmit particular viruses are capable of doing so.\n*   Similarly, not all plant viruses can be transmitted by insects; some are transmitted through other means such as mechanical transmission or nematodes.\n*   The correct assertion is that most viruses have a specific relationship with their vectors, meaning that each virus is typically transmitted by a specific type of insect or vector.\n\nAnswer: D'
│   │   },
│   │   {
│   │   │   'generated_answer': "The petioles of this rhubarb are splitting, and we need to determine which of the listed issues would not be the cause. \n\nFirst, let's consider physiological problems (A). Rhubarb is a hardy plant, but it can still experience physiological issues due to factors like temperature fluctuations, water stress, or nutrient deficiencies. These issues could potentially cause the petioles to split.\n\nNext, let's look at phytoplasma infection (B). Phytoplasmas are bacteria-like organisms that can infect plants, causing a range of symptoms including yellowing or browning of leaves, stunted growth, and distorted or split petioles. So, phytoplasma infection could also be a possible cause.\n\nNow, let's consider animal damage (D). Animals like rabbits, deer, or rodents might feed on the rhubarb leaves, causing damage to the petioles and potentially leading to splitting.\n\nFinally, let's think about bacteria (E). Bacterial infections can cause a range of symptoms in plants, including soft rot, lea

#### 1.2. Running SimpleQA
- We will use a pre-processed SimpleQA dataset from [llamastack/evals](https://huggingface.co/datasets/llamastack/evals/viewer/evals__simpleqa) which is obtained by transforming the input query into correct format accepted by `inference/chat-completion` API.
- Since we will be using this same dataset in our next example for Agentic evaluation, we will register it using the `/datasets` API, and interact with it through `/datasetio` API.

In [ ]:
simpleqa_dataset_id = "huggingface::simpleqa"

_ = client.datasets.register(
    dataset_id=simpleqa_dataset_id,
    provider_id="huggingface",
    url={"uri": "https://huggingface.co/datasets/llamastack/evals"},
    metadata={
        "path": "llamastack/evals",
        "name": "evals__simpleqa",
        "split": "train",
    },
    dataset_schema={
        "input_query": {"type": "string"},
        "expected_answer": {"type": "string"},
        "chat_completion_input": {"type": "chat_completion_input"},
    },
)


In [ ]:
eval_rows = client.datasetio.get_rows_paginated(
    dataset_id=simpleqa_dataset_id,
    rows_in_page=5,
)


In [ ]:
client.eval_tasks.register(
    eval_task_id="meta-reference::simpleqa",
    dataset_id=simpleqa_dataset_id,
    scoring_functions=["llm-as-judge::405b-simpleqa"],
)

response = client.eval.evaluate_rows(
    task_id="meta-reference::simpleqa",
    input_rows=eval_rows.rows,
    scoring_functions=["llm-as-judge::405b-simpleqa"],
    task_config={
        "type": "benchmark",
        "eval_candidate": {
            "type": "model",
            "model": "meta-llama/Llama-3.2-90B-Vision-Instruct",
            "sampling_params": {
                "strategy": {
                    "type": "greedy",
                },
                "max_tokens": 4096,
                "repeat_penalty": 1.0,
            },
        },
    },
)
pprint(response)


100%|██████████| 5/5 [00:48<00:00,  9.68s/it]


EvaluateResponse(
│   generations=[
│   │   {'generated_answer': 'The recipient of the IEEE Frank Rosenblatt Award in 2010 was Vladimir Vapnik'},
│   │   {
│   │   │   'generated_answer': "I am unable to verify who was awarded the Oceanography Society's Jerlov Award in 2018."
│   │   },
│   │   {
│   │   │   'generated_answer': "Radcliffe College was a women's liberal arts college, but it has since been integrated into Harvard University."
│   │   },
│   │   {
│   │   │   'generated_answer': "The Leipzig 1877 tournament was organized in the honor of 50th anniversary of the first chess club in Germany (the Leipzig Chess Club's) founding and of the 50th anniversary of Paul Morphy's birth"
│   │   },
│   │   {
│   │   │   'generated_answer': "Karl Küchler's 1908 guidebook states that Empress Elizabeth of Austria's favorite sculpture, which was made for her villa Achilleion at Corfu, depicted 'Dying Achilles'."
│   │   }
│   ],
│   scores={
│   │   'llm-as-judge::405b-simpleqa': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {'score': 'B', 'judge_feedback': 'B'},
│   │   │   │   {'score': 'C', 'judge_feedback': 'C'},
│   │   │   │   {'score': 'A', 'judge_feedback': 'A'},
│   │   │   │   {'score': 'B', 'judge_feedback': 'B'},
│   │   │   │   {'score': 'B', 'judge_feedback': 'B'}
│   │   │   ]
│   │   )
│   }
)

## 2. Agentic Evaluation

- In this example, we will demonstrate how to evaluate a agent candidate served by Llama Stack via `/agent` API.

- We will continue to use the SimpleQA dataset we used in previous example.

- Instead of running evaluation on model, we will run the evaluation on a Search Agent with access to search tool. We will define our agent evaluation candidate through `AgentConfig`.

> You will need to set the `TAVILY_SEARCH_API_KEY` in Secrets of this notebook.

In [ ]:
agent_config = {
    "model": "meta-llama/Llama-3.1-405B-Instruct",
    "instructions": "You are a helpful assistant",
    "sampling_params": {"strategy": {"type": "greedy"}},
    "tools": [
        {
            "type": "brave_search",
            "engine": "tavily",
            "api_key": userdata.get("TAVILY_SEARCH_API_KEY"),
        }
    ],
    "tool_choice": "auto",
    "tool_prompt_format": "json",
    "input_shields": [],
    "output_shields": [],
    "enable_session_persistence": False,
}

response = client.eval.evaluate_rows(
    task_id="meta-reference::simpleqa",
    input_rows=eval_rows.rows,
    scoring_functions=["llm-as-judge::405b-simpleqa"],
    task_config={
        "type": "benchmark",
        "eval_candidate": {
            "type": "agent",
            "config": agent_config,
        },
    },
)
pprint(response)


5it [00:26,  5.29s/it]


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'generated_answer': "I'm sorry but I cannot find the recipient of the IEEE Frank Rosenblatt Award in 2010."
│   │   },
│   │   {
│   │   │   'generated_answer': "I'm not sure who was awarded the Oceanography Society's Jerlov Award in 2018. Let me search for the information."
│   │   },
│   │   {
│   │   │   'generated_answer': "The women's liberal arts college in Cambridge, Massachusetts is called Radcliffe College. However, in 1999, it merged with Harvard University and is now known as the Radcliffe Institute for Advanced Study at Harvard University."
│   │   },
│   │   {
│   │   │   'generated_answer': 'The 1877 Leipzig tournament was organized in honor of Anderssen, a German chess master.'
│   │   },
│   │   {
│   │   │   'generated_answer': "Empress Elizabeth of Austria's favorite sculpture, made for her villa Achilleion at Corfu, depicted Achilles."
│   │   }
│   ],
│   scores={
│   │   'llm-as-judge::405b-simpleqa': ScoringResult(
│   │   │   aggregated_results={},
│   │   │   score_rows=[
│   │   │   │   {'score': 'C', 'judge_feedback': 'C.'},
│   │   │   │   {'score': 'C', 'judge_feedback': 'C'},
│   │   │   │   {'score': 'A', 'judge_feedback': 'A'},
│   │   │   │   {'score': 'A', 'judge_feedback': 'A'},
│   │   │   │   {'score': 'B', 'judge_feedback': 'B'}
│   │   │   ]
│   │   )
│   }
)